# Litmus image processing

**to use this notebook, first click the following link to run the notebook in google colab:**

<a target="_blank" href="https://colab.research.google.com/github/chengjz/litums_detection/blob/master/process_images.ipynb">Run in Google Colab</a><br><br>
This notebook will call the litmus_detection script on all of the photos you want to analyze in google colab. With this notebook, there is no need to install python3, opencv, etc on to your machine. 


---


### cell 1: mounting to drive
This cell "mounts" your notebook to drive so that you can access/work in google drive. This cell needs to be run once every session.

In [ ]:
from google.colab import drive
drive.mount('/content/drive',)



---

### cell 2: setting up your folders
this cell will create a project folder in drive and clone the repo from https://github.com/chengjz/litums_detection.git into that folder. Note that this particular cell only needs to be run once when initially setting up your folder system. It will only make the new folder and clone the repo if this has not already been done (so no worries if you accidentally run it again).<br>
**There are two folders for photos created here labeled: "cropped photos output" and "testing photos".**<br><br>

**to use the folder system:**
1. Navigate in your drive to the "testing photos" folder
2. Upload any photos to test into this folder 
3. Name the photos so that the concentration is listed first (i.e. **1.00**_any_additional_info.jpg)
4. the processed images (the wb frame and detected litmus squares) will be output into the "cropped photos output" folder

In [ ]:
import os
from os import path

# creating the project folder
if path.exists('/content/drive/MyDrive/project') == False:
  os.mkdir('/content/drive/MyDrive/project')
% cd /content/drive/MyDrive/project

# cloning the git repo into the project folder
if path.exists("/content/drive/MyDrive/project/litums_detection") == False:
  !  git init
  !  git clone https://github.com/chengjz/litums_detection.git
  !  git pull
# creating the testing photos and cropped photos output folders  
  os.mkdir('/content/drive/MyDrive/project/testing photos')
  os.mkdir('/content/drive/MyDrive/project/cropped photos output')



---
### cell 3: importing litmus_detection script

This cell imports the litmus_detection script and other neccessary moldues.

In [ ]:
#importing neccessary modules and the litmus detection script
% cd ..
% cd /content/drive/MyDrive/project/litums_detection
import sys  
path_to_module = "/content/drive/MyDrive/project/litums_detection"
sys.path.append(path_to_module)
import litmus_detection as ld
import cv2



---


### cell 4: Running the script
This cell will actually run the litmus detection script. This script will:
1. iterate through all of the photos in your testing photos folder
2. call the litmus detection script on each photo (and print out an error message if the code does not work for example if the QR code is not read)
3. output the wb strip and detected litmus squares to the cropped photos output folder
4. add the blue, green, and red values to a pandas data frame for all of the photos **(note that opencv order is blue, green, red. Not red, green, blue).**
5. print the final pandas dataframe and output this to an excel file in your main project folder

In [ ]:
import pandas as pd

# creating the free and total dataframes that will be filled with bgr values
free_df = pd.DataFrame({'free conc': [],'blue val': [],'green val': [],'red val': []})
total_df = pd.DataFrame({'total conc': [],'blue val': [],'green val': [],'red val': []})
index = 0

# iterating through all of the images in the testing photos folder
for root, dirs, files in os.walk("/content/drive/MyDrive/project/testing photos"):
  for fn in files:
    if fn.find(".jpg") > 0 or fn.find(".png") > 0:
      print(fn[:-4])
      file_Path = os.path.join(root, fn)
      try:
        frame = cv2.imread(file_Path)
        
        # calling the litmus detection script
        free_circle_cropped_img, free_bgr, total_circle_cropped_img, total_bgr, wb = ld.processing_img(frame)
        
        # outputing the wb strip and detected litmus squares to the cropped photos output folder
        cv2.imwrite("/content/drive/MyDrive/project/cropped photos output/"+fn[:-4]+"_free.jpg",free_circle_cropped_img)
        cv2.imwrite("/content/drive/MyDrive/project/cropped photos output/"+fn[:-4]+"_total.jpg",total_circle_cropped_img)
        cv2.imwrite("/content/drive/MyDrive/project/cropped photos output/"+fn[:-4]+"_wb_strip.jpg", wb)
        
        # adding the b, g and r values to the pandas dataframes
        free_df.loc[index] = [float(fn[:4])]+free_bgr
        total_df.loc[index] = [float(fn[:4])]+total_bgr
        index += 1
        print("----------------------------------------------------------")
      except:
        #if in an error occurs (such as QR code not being read)
        print(fn)
        print("error")
        print("----------------------------------------------------------")

# printing the final free and total dataframes and outputting it to excel   
print("free data frame")
print(free_df)
print("total data frame")
print(total_df)

with pd.ExcelWriter("/content/drive/MyDrive/project/output.xlsx",mode="w") as writer:
  free_df.to_excel(writer,sheet_name="free")
  total_df.to_excel(writer,sheet_name="total")
